In [3]:
import numpy as np
import pandas as pd
import matplotlib
from datetime import datetime, timedelta
import re
import pyarrow as pa

df = pd.DataFrame({'one': [-1, np.nan, 2.5],
                   'two': ['foo', 'bar', 'baz'],
                   'three': [True, False, True]},
                   index=list('abc'))


table = pa.Table.from_pandas(df)

In [4]:
import pyarrow.parquet as pq

pq.write_table(table, 'example.parquet')

^ Creates a single Parquet file. In practice, a Parquet dataset may consist of many files in many directories. We can read a single file back w/ read_table:

In [5]:
table2 = pq.read_table('example.parquet')
table2.to_pandas()

,one,two,three
a,-1.0,foo,True
b,NaN,bar,False
c,2.5,baz,True


Can pass a subset of columns to read, which can be much faster than reading the whole file (due to columnar layout):

In [6]:
pq.read_table('example.parquet', columns=['one', 'three'])

pyarrow.Table
one: double
three: bool
----
one: [[-1,null,2.5]]
three: [[true,false,true]]

When reading a subset of columns from a file that used a Pandas DF as source, we use read_pandas to maintain any additional index column data:

In [7]:
pq.read_pandas('example.parquet', columns=['two']).to_pandas()

,two
a,foo
b,bar
c,baz
